In [5]:
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.metrics import r2_score, mean_absolute_error, max_error, median_absolute_error
import joblib

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

In [2]:
file = 'players.csv'
data = pd.read_csv(file, delimiter=',')

data = data.drop('id', axis=1)
data = data.drop('name', axis=1)
data = data.drop('news_added', axis=1)

if 'birth_date' in data.columns:
    # Dátum konvertálása
    data['birth_date'] = pd.to_datetime(data['birth_date'], errors="coerce")
    
    # Mostani dátum (feltételezzük, hogy a predikció ideje 2025-ben van)
    today = pd.Timestamp("2025-05-20")
    
    # Életkor kiszámítása
    data['age'] = (today - data['birth_date']).dt.days // 365
else:
    data['age'] = None  # Ha nincs birth_date, létrehozunk egy üres oszlopot

data.head()

,now_cost,position,team,total_points,expected_goals_conceded_per_90,expected_goals_conceded,form,team_join_date,transfers_in,goals_conceded,saves_per_90,minutes,now_cost_rank_type,value_season,mng_underdog_draw,goals_conceded_per_90,penalties_missed,creativity,selected_rank,ict_index_rank_type,transfers_out_event,mng_clean_sheets,birth_date,starts,mng_goals_scored,penalties_saved,expected_goals,expected_assists_per_90,creativity_rank,mng_win,saves,clean_sheets,own_goals,bonus,chance_of_playing_next_round,ep_this,points_per_game_rank_type,ict_index,transfers_out,opta_code,direct_freekicks_order,clean_sheets_per_90,can_select,selected_by_percent,mng_underdog_win,corners_and_indirect_freekicks_order,cost_change_start_fall,dreamteam_count,points_per_game_rank,assists,expected_goal_involvements,event_points,points_per_game,yellow_cards,cost_change_event_fall,mng_draw,starts_per_90,threat_rank,cost_change_event,bps,selected_rank_type,influence_rank,penalties_order,in_dreamteam,red_cards,region,threat_rank_type,cost_change_start,value_form,chance_of_playing_this_round,form_rank,transfers_in_event,influence,web_name,news,mng_loss,creativity_rank_type,expected_goals_per_90,now_cost_rank,form_rank_type,ict_index_rank,influence_rank_type,ep_next,goals_scored,expected_goal_involvements_per_90,expected_assists,status,threat,age
0,54,MID,Arsenal,0,0.00,0.00,0.0,NaN,439,0,0.0,0,82,0.0,0,0.00,0,0.0,713,341,0,0,NaT,0,0,0,0.00,0.00,791,0,0,0,0,0,0.0,0.0,341,0.0,2825,p438098,NaN,0.00,False,0.0,0,NaN,1,0,790,0,0.00,0,0.0,0,0,0,0.00,788,0,0,294,791,NaN,False,0,NaN,340,-1,0.0,0.0,774,0,0.0,Fábio Vieira,Has joined Portuguese side FC Porto on loan fo...,0,341,0.00,133,332,791,341,0.0,0,0.00,0.00,u,0.0,NaN
1,65,FWD,Arsenal,42,0.87,5.82,0.0,2022-07-04,1238189,5,0.0,600,20,6.5,0,0.75,0,119.5,203,32,43,0,1997-04-03,6,0,0,3.05,0.08,263,0,0,2,0,6,0.0,0.0,26,52.6,1409397,p205651,NaN,0.30,True,1.0,0,NaN,5,2,217,2,3.57,0,2.5,4,0,0,0.90,126,0,152,31,340,NaN,False,0,30.0,32,-5,0.0,0.0,794,12,154.4,G.Jesus,Knee injury - Expected back 01 Jun,0,31,0.46,43,84,276,32,0.0,3,0.54,0.52,i,255.0,28.0
2,62,DEF,Arsenal,117,0.88,23.09,0.0,2020-09-01,5092906,22,0.0,2363,5,18.9,0,0.84,0,208.8,33,28,2165,0,1997-12-19,28,0,0,2.70,0.05,193,0,0,10,0,9,0.0,0.0,2,108.2,5398385,p226597,NaN,0.38,True,12.4,0,NaN,-2,2,35,2,3.91,0,4.2,4,0,0,1.07,110,0,459,12,80,NaN,False,0,30.0,9,2,0.0,0.0,791,23,584.6,Gabriel,Hamstring injury - Expected back 01 Jun,0,43,0.10,61,263,120,30,0.0,3,0.15,1.21,i,287.0,27.0
3,77,FWD,Arsenal,96,0.93,19.00,0.0,2023-06-28,2589815,20,0.0,1839,4,12.5,0,0.98,0,257.0,66,16,138,0,1999-06-11,21,0,0,9.39,0.08,157,0,0,7,0,14,25.0,0.0,8,142.6,3526422,p219847,NaN,0.34,True,5.7,0,NaN,3,1,20,3,11.02,0,4.6,4,0,0,1.03,34,0,340,12,144,3.0,False,0,80.0,15,-3,0.0,25.0,798,22,467.2,Havertz,Hamstring injury - 25% chance of playing,0,17,0.46,14,87,75,16,0.1,9,0.54,1.63,d,701.0,25.0
4,40,GKP,Arsenal,0,0.00,0.00,0.0,NaN,0,0,0.0,0,54,0.0,0,0.00,0,0.0,642,68,1,0,NaT,0,0,0,0.00,0.00,594,0,0,0,0,0,0.0,0.0,68,0.0,4620,p463748,NaN,0.00,False,0.0,0,NaN,0,0,615,0,0.00,0,0.0,0,0,0,0.00,555,0,0,71,607,NaN,False,0,NaN,38,0,0.0,0.0,489,0,0.0,Hein,Loaned to Real Valladolid,0,57,0.00,668,59,607,68,0.0,0,0.00,0.00,u,0.0,NaN


In [4]:
# Csak nem kapus pozíciók
positions = ["GKP", "DEF", "MID", "FWD"]

# Dictionary a modellek tárolására
position_models = {}

# Külön modell minden pozícióra
for pos in positions:
    df_pos = data[
        (data["position"] == pos) &
        (data["expected_goals"].notna()) &
        (data["age"].notna()) &
        (data["creativity"].notna())
    ][["position", "team", "now_cost", "age", "creativity", "expected_goals"]]
    
    X = df_pos[["position", "team", "now_cost", "age", "creativity"]]
    y = df_pos["expected_goals"]
    
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
    
    # Pipeline létrehozása
    categorical_features = ["position", "team"]
    numerical_features = ["now_cost", "age", "creativity"]

    categorical_transformer = Pipeline(steps=[
        ("imputer", SimpleImputer(strategy="most_frequent")),
        ("onehot", OneHotEncoder(handle_unknown="ignore"))
    ])

    numerical_transformer = Pipeline(steps=[
        ("imputer", SimpleImputer(strategy="mean"))
    ])

    preprocessor = ColumnTransformer(
        transformers=[
            ("cat", categorical_transformer, categorical_features),
            ("num", numerical_transformer, numerical_features)
        ]
    )

    model = Pipeline(steps=[
        ("preprocessor", preprocessor),
        ("regressor", RandomForestRegressor(n_estimators=100, random_state=42))
    ])

    model.fit(X_train, y_train)
    position_models[pos] = model

# Mentés fájlba: külön fájl minden pozícióhoz
saved_models = []
for pos, model in position_models.items():
    path = f"model{pos.lower()}.pkl"
    joblib.dump(model, path)
    saved_models.append(path)

saved_models

['modelgkp.pkl', 'modeldef.pkl', 'modelmid.pkl', 'modelfwd.pkl']

In [ ]:
y_pred = model.predict(X_test)

r2 = r2_score(y_test, y_pred)
mae = mean_absolute_error(y_test, y_pred)
max_err = max_error(y_test, y_pred)
medae = median_absolute_error(y_test, y_pred)

print("R^2:", r2)
print("Mean Absolute Error (MAE):", mae)
print("Max Error:", max_err)
print("Median Absolute Error (MedAE):", medae)  

R^2: 0.5367686710597181
Mean Absolute Error (MAE): 1.0781071428571434
Max Error: 5.561299999999998
Median Absolute Error (MedAE): 0.5265500000000003
